# Initialize notebook

This notebook will be run by other notebooks in order to initialze helpful libraries, functions and variables.

In [3]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [1]:
from wurst import *
from wurst.searching import *
from matplotlib import pyplot as plt
from brightway2 import *
import numpy as np
import pyprind
import seaborn as sns
import pandas as pd
# import winsound
import math
from scipy import stats
import pickle
import collections

In [2]:
projects.set_current("REMIND_ecoinvent")

In [4]:
try: ei = databases["ecoinvent_3.5"]
except: pass

In [7]:
plt.style.use('seaborn')

#define standard color palette:
sns.set_palette('Set1',9)

#create longer color list for complex figures
colors=[]
colors.extend(sns.color_palette('Set1',9))
colors.extend(sns.color_palette('Dark2',8))

colors.extend(sns.color_palette('Set2',8))
colors.extend(sns.color_palette('Accent',8))

colors.extend(colors)
colors.extend(colors)
colors.extend(colors)
colors.extend(colors)

In [8]:
#Here we create some dictionaries of LCIA methods:

lcia_methods={
    #'CC_non_fossil':('IPCC 2013', 'climate change', 'GWP 100a', 'Complete'),
    'CC':('IPCC 2013', 'climate change', 'GWP 100a', 'CO2 storage'),
    'TA':('ReCiPe Midpoint (H)', 'terrestrial acidification', 'TAP100'),
    'POF':('ReCiPe Midpoint (H)','photochemical oxidant formation','POFP'),
    'PMF':('ReCiPe Midpoint (H)', 'particulate matter formation', 'PMFP'),
    'MD':('ReCiPe Midpoint (H)', 'metal depletion', 'MDP'),
    'HT':('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf'),
    'MET':('ReCiPe Midpoint (H)', 'marine ecotoxicity', 'METPinf'),
    'ME':('ReCiPe Midpoint (H)', 'marine eutrophication', 'MEP'),
    'FD':('ReCiPe Midpoint (H)', 'fossil depletion', 'FDP'),
    'IR':('ReCiPe Midpoint (H)', 'ionising radiation', 'IRP_HE'),
    'OD':('ReCiPe Midpoint (H)', 'ozone depletion', 'ODPinf'),
    'FET':('ReCiPe Midpoint (H)', 'freshwater ecotoxicity', 'FETPinf'),
    'TET':('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf'),
    'ALO':('ReCiPe Midpoint (H)', 'agricultural land occupation', 'ALOP'),
    'NLT':('ReCiPe Midpoint (H)', 'natural land transformation', 'NLTP'),
    'ULO':('ReCiPe Midpoint (H)', 'urban land occupation', 'ULOP'),
    'WD':('ReCiPe Midpoint (H)', 'water depletion', 'WDP'),
    'FE':('ReCiPe Midpoint (H)', 'freshwater eutrophication', 'FEP'),
    'R_HH' : ('ReCiPe Endpoint (H,A)', 'human health', 'total', 'CO2 storage'),
    'R_EQ' : ('ReCiPe Endpoint (H,A)', 'ecosystem quality', 'total', 'CO2 storage'),
    'R_R' : ('ReCiPe Endpoint (H,A)', 'resources', 'total'),
    'R_Total' : ('ReCiPe Endpoint (H,A)', 'total', 'total', 'CO2 storage'),
    
    'CEDB': ('cumulative energy demand','biomass','renewable energy resources, biomass'),
      'CEDF': ('cumulative energy demand','fossil','non-renewable energy resources, fossil'),
      'CEDG': ('cumulative energy demand','geothermal','renewable energy resources, geothermal, converted'),
      'CEDN': ('cumulative energy demand','nuclear','non-renewable energy resources, nuclear'),
      'CEDFr': ('cumulative energy demand','primary forest','non-renewable energy resources, primary forest'),
      'CEDS': ('cumulative energy demand','solar','renewable energy resources, solar, converted'),
      'CEDH': ('cumulative energy demand','water','renewable energy resources, potential (in barrage water), converted'),
      'CEDW': ('cumulative energy demand','wind','renewable energy resources, kinetic (in wind), converted')

    }

lcia_methods_short={
    'CC':('IPCC 2013', 'climate change', 'GWP 100a', 'CO2 storage'),
    'HT':('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf'),
    'POF':('ReCiPe Midpoint (H)','photochemical oxidant formation','POFP'),
    'PMF':('ReCiPe Midpoint (H)', 'particulate matter formation', 'PMFP'),
    'MD':('ReCiPe Midpoint (H)', 'metal depletion', 'MDP'),
    'CEDB': ('cumulative energy demand','biomass','renewable energy resources, biomass'),
      'CEDF': ('cumulative energy demand','fossil','non-renewable energy resources, fossil'),
      'CEDG': ('cumulative energy demand','geothermal','renewable energy resources, geothermal, converted'),
      'CEDN': ('cumulative energy demand','nuclear','non-renewable energy resources, nuclear'),
      'CEDFr': ('cumulative energy demand','primary forest','non-renewable energy resources, primary forest'),
      'CEDS': ('cumulative energy demand','solar','renewable energy resources, solar, converted'),
      'CEDH': ('cumulative energy demand','water','renewable energy resources, potential (in barrage water), converted'),
      'CEDW': ('cumulative energy demand','wind','renewable energy resources, kinetic (in wind), converted')
    }
  
titles={'CC': 'Climate Change', 
        'TA': 'Terrestrial Acidification',
        'POF':'Photochemical Oxidant Formation', 
        'PMF':'Particulate Matter Formation',
        'MD': 'Mineral Depletion',
        'HT':'Human Toxicity',
        'CED':'Cumulative Energy Demand',

        'MET':'Marine Ecotoxicity',
        'ME':'Marine Eutrophication',
        'FD':'Fossil Depletion',
        'IR': 'Ionising Radiation',
        'OD':'Ozone Depletion',
        'FET':'Freshwater Ecotoxicity',
        'TET':'Terrestrial Ecotoxicity',
        'ALO': 'Agricultural Land Occupation',
        'NLT': 'Natural Land Transformation',
        'ULO': 'Urban Land Occupation',
        'WD': 'Water Depletion',
        'FE': 'Freshwater Eutrophication',
        'R_HH':'ReCiPe Endpoint Human Health',
        'R_EQ':'ReCiPe Endpoint Ecosystem Quality',
        'R_R':'ReCiPe Endpoint Resources',
        'R_Total':'ReCiPe Endpoint Total'
       }

short_titles={'CC': 'Climate Change', 
        'TA': 'Terrestrial Acidification',
        'POF':'Photochemical Oxidant Formation', 
        'PMF':'Particulate Matter Formation',
        'MD': 'Mineral Depletion',
        'HT':'Human Toxicity',
        'CED':'Cumulative Energy',
        'MET':'Marine Ecotoxicity',
        'ME':'Marine Eutrophication',
        'FD':'Fossil Depletion',
        'IR': 'Ionising Radiation',
        'OD':'Ozone Depletion',
        'FET':'Freshwater Ecotoxicity',
        'TET':'Terrestrial Ecotoxicity',
        'ALO': 'Agricultural Land Occupation',
        'NLT': 'Natural Land Transformation',
        'ULO': 'Urban Land Occupation',
        'WD': 'Water Depletion',
        'FE': 'Freshwater Eutrophication',
        'R_HH':'Endpoint Human Health',
        'R_EQ':'Endpoint Ecosystem Quality',
        'R_R':'Endpoint Resources',
        'R_Total':'Endpoint Total'
       }

two_line_titles = {'ALO': 'Agricultural Land\nOccupation',
 'CED': 'Cumulative Energy\nDemand',
 'FD': 'Fossil Depletion',
 'FE': 'Freshwater Eutrophication',
 'FET': 'Freshwater Ecotoxicity',
 'CC': 'Climate Change',
 'HT': 'Human Toxicity',
 'IR': 'Ionising Radiation',
 'MD': 'Mineral Depletion',
 'ME': 'Marine Eutrophication',
 'MET': 'Marine Ecotoxicity',
 'NLT': 'Natural Land\nTransformation',
 'OD': 'Ozone Depletion',
 'PMF': 'Particulate Matter\nFormation',
 'POF': 'Photochemical Oxidant\nFormation',
 'R_EQ': 'ReCiPe Endpoint\nEcosystem Quality',
 'R_HH': 'ReCiPe Endpoint\nHuman Health',
 'R_R': 'ReCiPe Endpoint\nResources',
 'R_Total': 'ReCiPe Endpoin\nTotal',
 'TA': 'Terrestrial Acidification',
 'TET': 'Terrestrial Ecotoxicity',
 'ULO': 'Urban Land\nOccupation',
 'WD': 'Water Depletion'}

units={'CC': 'kg CO$_2$ eq', 
        'TA': 'kg SO$_2$ eq',
        'POF':'kg NMVOC', 
        'PMF':'kg PM$_{10}$ eq',
        'MD': 'kg Fe eq',
        'HT':'kg 1,4 DB eq',
        'CED':'MJ',
        'MET':'kg 14-DCB eq',
        'ME': 'kg N eq',
        'FD':'kg oil eq',
        'IR': 'kg U235 eq',
        'OD':'kg CFC11 eq',
        'FET':'kg 14-DCB eq',
        'TET':'kg 14-DCB eq',
        'ALO': 'm$^2$yr',
        'NLT': 'm$^2$',
        'ULO': 'm$^2$yr',
        'WD': 'm$^3$ H$_2$O',
        'FE': 'kg P eq',
        'R_HH':'Points',
        'R_EQ':'Points',
        'R_R':'Points',
        'R_Total':'Points'
        }
               
titles_units={}
for cat in titles.keys():
    titles_units[cat]=titles[cat] + ' (' + units[cat] + ')'
    
cats_units={}
for cat in titles.keys():
    cats_units[cat]=cat + ' (' + units[cat] + ')'

In [9]:
def LCA_to_df(datasets, cats=['CC', 'R_Total'], amount=1, names=['name', 'location']):
    # calcuate a LCA for a list of datasets for a list of methods and return a pandas dataframe:
    results = {}
    index_dict={}
    for ds in datasets:
        index_dict[ds['code']]= tuple(ds[i] for i in names)
    
    for cat in cats:
        print(cat)
        lca.switch_method(lcia_methods[cat])
        results[cat]={}
        for dataset in pyprind.prog_bar(datasets):   
        #for dataset in datasets:
                    lca.redo_lcia({dataset: amount})
                    results[cat][dataset['code']]  = lca.score
                    
    # We group all energy into one category:    
    for cat in ['CEDF', 'CEDB','CEDG','CEDN','CEDFr','CEDS','CEDH','CEDW']:
        if cat in results.keys():  
            for key in results[cat].keys():
                if 'CED' not in results.keys(): results['CED'] = {}
                
                try: results['CED'][key]+=results[cat][key]
                except KeyError: results['CED'][key] = results[cat][key]
            del results[cat]
   
    return pd.DataFrame(results).rename(index=index_dict)

In [10]:
def contribution_LCA_to_df(datasets, cats=['CC', 'R_Total'], amount=1, names=['name', 'location']):
    #calculate foreground contribution LCA of a list of datasets and return a multi-index dataframe:
    results = {}
    codes={}
    index_dict={}
    for ds in datasets:
        index_dict[ds['code']]= tuple(ds[i] for i in names)  
    
    for cat in cats:
        lca.switch_method(lcia_methods[cat])
        cf_dict = dict(Method(lcia_methods[cat]).load())
        codes[cat]={}
        results[cat]={}
        for dataset in datasets:      
            for exc in dataset.technosphere():
                existing_value = 0
                if (dataset['code'],exc.input['name']) in results[cat].keys(): existing_value= results[cat][(dataset['code'],exc.input['name'])] 
                if exc['amount'] == 0: continue
                if exc['input'] in codes[cat]:
                    results[cat][(dataset['code'],exc.input['name'])] = amount*codes[cat][exc['input']]*exc['amount'] + existing_value
                else:
                    lca.redo_lcia({exc.input: exc['amount']})
                    results[cat][(dataset['code'],exc.input['name'])]  = lca.score*amount + existing_value
                    codes[cat][exc['input']] = (lca.score/exc['amount'])
    
            for exc in dataset.biosphere():    
                if exc.input in cf_dict: # Not all flows are characterized
                    existing_value = 0
                    if (dataset['code'],exc.input['name']) in results[cat].keys(): existing_value= results[cat][(dataset['code'],exc.input['name'])] 
                    results[cat][(dataset['code'],exc.input['name'])]  = amount*exc['amount'] * cf_dict[exc.input] + existing_value
                    
    
    for cat in ['CEDF', 'CEDB','CEDG','CEDN','CEDFr','CEDS','CEDH','CEDW']:
        if cat in results.keys():  
            for key in results[cat].keys():
                if 'CED' not in results.keys(): results['CED'] = {}
                
                try: results['CED'][key]+=results[cat][key]
                except KeyError: results['CED'][key] = results[cat][key]
            del results[cat]
                   
    return pd.DataFrame(results).unstack().sort_index(axis=1).rename(index=index_dict)

In [1]:
def get_biosphere_factors(flows, cats=['GWP', 'R_Total']):
    # calcuate a LCA for a dict of biosphere flows for a list of methods and return a pandas dataframe:
    results = {}
   
    for cat in cats:
        lca.switch_method(lcia_methods[cat])
        cf_dict = dict(Method(lcia_methods[cat]).load())
        results[cat]={}
        
        for name, exc in flows.items():    
            if ('biosphere3', exc['code']) in cf_dict: # Not all flows are characterized
                results[cat][name] = cf_dict[('biosphere3', exc['code'])]
                    
    # We usually prefer to group fossil and nuclear non renewable energy into one category called non-renewable energy:    
    if 'CEDF' in results.keys():
        #print(results.keys())
        results['CED']=results['CEDF']
        del results['CEDF']
    
    for cat in ['CEDF', 'CEDB','CEDG','CEDN','CEDFr','CEDS','CEDH','CEDW']:
        if cat in results.keys():  
            for key in results[cat].keys():
                if 'CED' not in results.keys(): results['CED'] = {}
                
                try: results['CED'][key]+=results[cat][key]
                except KeyError: results['CED'][key] = results[cat][key]
            del results[cat]
                   
    return pd.DataFrame(results)

In [11]:
def group_into_other(df, lim=0.01):
    #Group all processes that contribute less than a specified amount to the results for each LCIA method into 'other'. 
    norm=df.divide(df.sum(axis=1,level=0), axis=1, level=0).fillna(0)
    other_list=[]
    for col in norm.columns:
        if norm[col].max() < lim and np.absolute(norm[col].min()) < lim: 
            other_list.append(col)
    other=df[other_list].sum(axis=1,level=0)
    other.columns = pd.MultiIndex.from_product([other.columns, ['other']])
    return pd.concat([df.drop(other_list,axis=1),other], axis=1).sort_index(axis=1)

In [12]:
def cm2in(*tupl):
    inch = 2.54
    if isinstance(tupl[0], tuple):
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)

In [5]:
def import_karma(path_to_karma="../data/lci-Carma-CCS.xlsx"):
    db_name = "Carma CCS"   
    if db_name not in databases:
            
        sp = ExcelImporter(path_to_karma)
        sp.apply_strategies()
        sp.match_database(fields=["name", "unit", "location"]) 
        sp.match_database('ecoinvent_3.5',  fields=["reference product", "name", "unit", "location"]) 
        sp.match_database('ecoinvent_3.5',  fields=[ "name", "unit", "location"])
        sp.write_database()
        del sp
    else:
        print("Database {} already present.".format(db_name))